In [ ]:
import nest_asyncio
import asyncio
import concurrent.futures

import socket
import socketserver
import sys

import json
from json import JSONDecodeError
import datetime
import ipaddress
import functools
import numpy as np
import pandas as pd

import IPython.display

import pyspark
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as func

nest_asyncio.apply()

class PacketServer(socketserver.BaseRequestHandler):

    def _get_hostnames_if_necessary(self):
        if not hasattr(self, "_hostnames_init"):
            self._update_hostnames()
            self._hostnames_init = True
            self._hostnames_sent = datetime.datetime.now()
            return self._get_hostnames()
        if (datetime.datetime.now() - self._hostnames_sent).total_seconds() < 30:
            return None
        self._hostnames_sent = datetime.datetime.now()
        return self._get_hostnames()

    def _extract_host(self, line):
        sline = line.rstrip().split("\t")
        return (sline[0], sline[-1])

    def _update_hostnames(self):
        with open("/etc/lx2-hosts", "r") as f:
            self._hostnames = dict(self._extract_host(line) for line in f)
        self._hostnames_updated = datetime.datetime.now()

    def _get_hostnames(self):
        if (datetime.datetime.now() - self._hostnames_updated).total_seconds() > 300:
            self._update_hostnames()
        return self._hostnames

    def handle(self):
        streamer = iter(self._stream_packets(self._parse_message))
        while True:
            p = next(streamer)
            hostnames = self._get_hostnames_if_necessary()
            if hostnames != None:
                hostnames_out = {
                    "hostname_map": hostnames
                }
                p.insert(0, hostnames_out)
            for m in p:
                json_out = json.dumps(m, separators=(",", ":")) + "\n"
                self.request.sendall(
                    bytes(
                        json_out, 
                        "utf-8"
                    )
                )

    def _stream_packets(self, _parse):
        with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
            s.bind(("0.0.0.0", 22055))
            while True:
                p, _ = s.recvfrom(65536)
                parsed = ""
                while parsed == "":
                    try:
                        parsed = _parse(p)
                    except JSONDecodeError:
                        parsed = ""
                        pn, _ = s.recvfrom(65536)
                        p += pn
                yield parsed

    def _parse_message(self, x):
        s = x.decode("utf-8").rstrip()
        return [json.loads(m) for m in s.split("\n")]

def serve_packets():
    with socketserver.TCPServer(("127.0.0.1", 11111), PacketServer) as server:
        server.serve_forever()
        
record_handle = asyncio.new_event_loop().run_in_executor(
    concurrent.futures.ThreadPoolExecutor(max_workers=1),
    serve_packets
)

spark = SparkSession.builder\
    .appName("Stream Socket")\
    .master("local[*]")\
    .enableHiveSupport()\
    .getOrCreate()

sc = spark.sparkContext
ssc = StreamingContext(sc, 1)
ssc.checkpoint("/tmp/streamsocket")

def error_record(x):
    x.update({"record_type": "error"})
    return x


def is_addr_local(saddr):
    addr = ipaddress.IPv4Address(saddr)
    if not addr.is_private:
        return False
    if addr in ipaddress.IPv4Network("10.0.0.0/8"):
        return True
    if addr in ipaddress.IPv4Network("192.168.0.0/16"):
        return True
    if addr in ipaddress.IPv4Network("172.16.0.0/12"):
        return True
    return False


def extract_cflow_payload(cflow_record):
    src_port = int(cflow_record.get("text_cflow_srcport", "0"))
    dst_port = int(cflow_record.get("text_cflow_dstport", "0"))
    flow_packets = int(cflow_record.get("text_cflow_packets", "0"))
    flow_bytes = int(cflow_record.get("text_cflow_octets", "0"))
    flow_direction = int(cflow_record.get("text_cflow_direction", "0"))
    flow_timestart = cflow_record.get("cflow_timedelta_cflow_timestart", "0")
    flow_timeend = cflow_record.get("cflow_timedelta_cflow_timeend", "0")
    src_is_client = False
    dst_is_client = False
    src_addr = cflow_record["text_cflow_srcaddr"]
    dst_addr = cflow_record["text_cflow_dstaddr"]
    src_is_private = is_addr_local(src_addr)
    dst_is_private = is_addr_local(dst_addr)
    src_is_client = src_is_private and not dst_is_private
    dst_is_client = dst_is_private and not src_is_private
    if src_is_client == dst_is_client:
        dst_is_client = src_port < dst_port and dst_is_private
        src_is_client = dst_port < src_port and src_is_private
        if src_is_client == dst_is_client:
            dst_is_client = src_port < dst_port
            src_is_client = dst_port < src_port
            if src_is_client == dst_is_client:
                src_is_client = flow_direction == 0
                dst_is_client = not src_is_client
    if src_is_client:
        client_addr = src_addr
        server_addr = dst_addr
        client_port = src_port
        server_port = dst_port
    if dst_is_client:
        client_addr = dst_addr
        server_addr = src_addr
        client_port = dst_port
        server_port = src_port      
    cflow_payload = {
        "flow_packets": flow_packets,
        "flow_bytes": flow_bytes,
        "flow_direction": flow_direction,
        "flow_timestart": flow_timestart,
        "flow_timeend": flow_timeend,
        "client_addr": client_addr,
        "server_addr": server_addr,
        "client_port": client_port,
        "server_port": server_port,
        "src_addr": src_addr,
        "dst_addr": dst_addr,
        "src_port": src_port,
        "dst_port": dst_port
    }
    return cflow_payload


def extract_dns_payload(dns_record, ip_record):
    client_addr = ip_record["ip_ip_dst"]
    dns_query_name = dns_record["text_dns_qry_name"]
    dns_resp_types = dns_record["text_dns_resp_type"]
    dns_rcode = dns_record["dns_flags_dns_flags_rcode"]
    dns_payload = {
        "client_addr": client_addr,
        "dns_query_name": dns_query_name,
        "dns_resp_types": dns_resp_types,
        "dns_rcode": dns_rcode
    }
    return dns_payload


def parse_record(x):
    
    this_record = {"payload": {"_original_record": x}}
    records = []
    errored = False
    
    try:
        this_record.update(json.loads(x))
    except JSONDecodeError:
        errored = True
        records.append(error_record(this_record))
    
    if "hostname_map" in this_record:
        hostname_map = this_record.get("hostname_map")
        for (client_addr, client_hostname) in hostname_map.items():
            this_hostname_map_record = {
                "record_type": "hostname_map",
                "payload": {
                    "hostname_map": {
                        "client_addr": client_addr,
                        "client_hostname": client_hostname
                    }
                }
            }
            records.append(this_hostname_map_record)
    elif "cflow" in this_record.get("layers", {}):
        if "text_cflow_srcaddr" not in this_record["layers"]["cflow"].keys():
            this_record.update({"record_type": "cflow_template"})
            records.append(this_record)
        else:
            this_record.update({"record_type": "cflow"})
            cflow_record = this_record.get("layers")\
                .get("cflow")

            if type(cflow_record["text_text"]) is list:
                nflows = len(cflow_record["text_text"])
                list_fields_this = [
                    k for (k, v) in cflow_record.items()
                    if type(v) is list
                ]
                flow_fields_this = [
                    k for k in list_fields_this
                    if len(cflow_record[k]) == nflows
                ]
                for field_values in zip(
                    *[cflow_record[f] for f in flow_fields_this]
                ):
                    this_cflow_record = {
                        **this_record,
                        "payload": {
                            "cflow": extract_cflow_payload({
                                k: v for (k, v) in zip(
                                    flow_fields_this, 
                                    field_values
                                )
                            })
                        }
                    }
                    records.append(this_cflow_record)
            elif type(cflow_record["text_text"]) is str:
                this_cflow_record = {
                    **this_record,
                    "payload": {
                        "cflow": extract_cflow_payload(cflow_record)
                    }
                }
                records.append(this_cflow_record)
            elif not errored: 
                errored = True
                records.append(error_record(this_record))
        
    elif "dns" in this_record.get("layers", {}):
        this_record.update({
            "record_type": "dns",
            "payload": {
                "dns" : extract_dns_payload(
                    this_record.get("layers").get("dns"),
                    this_record.get("layers").get("ip")
                )
            }
        })
        records.append(this_record)
    elif not errored: 
        errored = True
        records.append(error_record(this_record))
    
    for out in records:
        yield out

        
def filter_record(x):
    return x["record_type"] in set(["hostname_map", "cflow", "dns"])


def update_client_addr(part, client_addr_state):

    now = datetime.datetime.now()
    if client_addr_state != None:
        recent_update = datetime.datetime.fromisoformat(client_addr_state["client_recent_update"])
        delta_seconds_recent = (now - recent_update).total_seconds()
        for (k, v) in client_addr_state.items():
            if k.startswith("counts"):
                for (n, c) in v.items():
                    client_addr_state[k][n] = c - 0.3*c*(1 - np.exp(-delta_seconds_recent/300))
    if client_addr_state is None:
        client_addr_state = {
            "client_recent_update": now.isoformat(),
            "client_hostname": "",
            "counts_record_type": {},
            "counts_dns_query_name": {},
            "counts_server_addr": {},
            "counts_server_port": {}
        }    
    for x in part:
            
        this_record_type = x["record_type"]
        counts_this_record_type = client_addr_state["counts_record_type"]\
            .get(this_record_type, 0) + 1
        client_addr_state["counts_record_type"][this_record_type] = counts_this_record_type
        if x["record_type"] == "hostname_map":
            client_addr_state["client_hostname"] = x["payload"]["hostname_map"]["client_hostname"]
        elif x["record_type"] == "dns":
            this_dns_query_name = x["payload"]["dns"]["dns_query_name"]
            counts_this_dns_query_name = client_addr_state["counts_dns_query_name"]\
                .get(this_dns_query_name, 0) + 1
            client_addr_state["counts_dns_query_name"][this_dns_query_name] = counts_this_dns_query_name
            client_addr_state["client_recent_update"] = datetime.datetime.now().isoformat()
        elif x["record_type"] == "cflow":
            this_server_addr = x["payload"]["cflow"]["server_addr"]
            counts_this_server_addr = client_addr_state["counts_server_addr"]\
                .get(this_server_addr, 0) + 1
            client_addr_state["counts_server_addr"][this_server_addr] = counts_this_server_addr
            this_server_port = x["payload"]["cflow"]["server_port"]
            counts_this_server_port = client_addr_state["counts_server_port"]\
                .get(this_server_port, 0) + 1
            client_addr_state["counts_server_port"][this_server_port] = counts_this_server_port
            client_addr_state["client_recent_update"] = datetime.datetime.now().isoformat()

    for (k, v) in client_addr_state.items():
        if k.startswith("counts"):
            keys_to_delete = []
            for (n, c) in v.items():
                if c < 0.8:
                    keys_to_delete.append(n)
            for m in keys_to_delete:
                del v[m]
        
    return client_addr_state
        
    
packets = ssc.socketTextStream("127.0.0.1", 11111)\
    .flatMap(parse_record)\
    .filter(filter_record)


def map_to_client_addr_tuple(x):
    record_type = x["record_type"]
    return (x["payload"][record_type]["client_addr"], x)


packets_client_addr = packets.map(map_to_client_addr_tuple)\
    .updateStateByKey(update_client_addr)\
    .filter(lambda x: sum([x[1]["counts_record_type"].get(t, 0) for t in ["dns", "cflow"]]) > 0)

def format_record(x):
    out = {}
    for (k, v) in x.items():
        if k.startswith("counts"):
            out[k] = " ".join([str(d) for d in sorted(v, key=v.get, reverse=True)])
        else:
            out[k] = v
    return out

def display_dataframe(x):
    raw = x.map(lambda r: r[1])\
        .map(format_record)\
        .collect()
    df = pd.DataFrame.from_dict(raw)
    if "client_hostname" not in df.columns.values:
        return df
    df.set_index("client_hostname", inplace=True)
    df.sort_index(inplace=True)
    del df["counts_record_type"]
    return df

display_handle = IPython.display.display(display_id=True)
packets_client_addr.foreachRDD(
    lambda x: display_handle.update(display_dataframe(x))
)
ssc.start()
ssc.awaitTermination()

,client_recent_update,counts_dns_query_name,counts_server_addr,counts_server_port
client_hostname,,,,
EntrywayLamp,2020-07-25T20:42:37.246602,,34.224.125.111,443
FamilyDesktopPC,2020-07-25T20:42:45.235422,array615.prod.do.dsp.mp.microsoft.com,192.168.1.10 192.168.1.50 52.242.211.89 52.200...,54524 443 53 80
JimsMacBook,2020-07-25T20:42:48.235274,browser.pipe.aria.microsoft.com lb._dns-sd._ud...,192.168.1.10 151.101.196.133 192.168.1.50 52.2...,443 53 993 80 5228
JimsiPad,2020-07-25T20:42:48.234871,,192.168.1.50 17.253.26.253 17.253.4.125 17.253...,53 123 443
JimsiPhone,2020-07-25T20:42:34.198874,,192.168.1.50 184.28.188.187 17.57.144.22 74.12...,443 53 993 5223
KidsBedroomAppleTV,2020-07-25T20:42:01.223481,,17.188.166.187 17.57.144.20,3493 5223 443
KidsBedroomSmallDisplay,2020-07-25T20:42:48.234630,www.google.com,10.15.5.1 8.8.8.8 142.250.68.99 192.168.1.50 1...,0 443 53
LitterRobot,2020-07-25T20:42:19.239281,,54.167.10.56,2001
LivingRoomAppleTV,2020-07-25T20:42:45.208214,,74.125.170.75 172.217.11.78 172.217.14.110 17....,443 5223 3493 3485 3484 3488
